In [2]:
!pip install geopandas --upgrade --q
!pip install gtfs_kit --upgrade --q

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 15.4 MB 36 kB/s 
     |████████████████████████████████| 6.3 MB 59.9 MB/s 
     |████████████████████████████████| 62 kB 615 kB/s 
     |████████████████████████████████| 994 kB 20.6 MB/s 
     |████████████████████████████████| 10.0 MB 16.3 MB/s 


In [3]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'content/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [43]:
# Read feed and describe/content/NorthernKentucky_TANK.zip

path = DATA_DIR/'gtfs (2).zip'
feed = gk.read_feed(path, dist_units='km')
print(feed)

* agency --------------------
	  agency_id  ...                              agency_fare_url
0       466  ...  http://128bc.org/rev-hartwell-area-shuttle/

[1 rows x 7 columns]
* calendar --------------------
	          service_id                         service_name  ...  start_date  end_date
0  c_1725_b_642_d_31  Year-round calendar (Monday-Friday)  ...    20151101  20190101

[1 rows x 11 columns]
* calendar_dates --------------------
	          service_id      date            holiday_name  exception_type
0  c_1725_b_642_d_31  20190101          New Year's Day               2
1  c_1725_b_642_d_31  20181225           Christmas Day               2
2  c_1725_b_642_d_31  20181123  Day after Thanksgiving               2
3  c_1725_b_642_d_31  20181122        Thanksgiving Day               2
4  c_1725_b_642_d_31  20180704        Independence Day               2
* fare_attributes --------------------
	None
* fare_rules --------------------
	None
* feed_info --------------------
	             

In [44]:
feed.describe()

,indicator,value
0,agencies,[128 Business Council]
1,timezone,America/New_York
2,start_date,20151101
3,end_date,20190101
4,num_routes,9
5,num_trips,74
6,num_stops,64
7,num_shapes,44
8,sample_date,20151105
9,num_routes_active_on_sample_date,9


In [45]:
feed.validate()

,type,message,table,rows
0,warning,Unrecognized column service_name,calendar,[]
1,warning,Feed expired,calendar/calendar_dates,[]
2,warning,Unrecognized column holiday_name,calendar_dates,[]
3,warning,Unrecognized column feed_license,feed_info,[]
4,warning,Unrecognized column feed_contact_email,feed_info,[]
5,warning,Unrecognized column feed_contact_url,feed_info,[]
6,warning,Unrecognized column route_sort_order,routes,[]
7,warning,Unrecognized column min_headway_minutes,routes,[]
11,warning,Unrecognized column continuous_stops,stop_times,[]
12,warning,Unrecognized column area_id,stop_times,[]


In [46]:
# List feed
gk.list_feed(path)

,file_name,file_size
0,fare_attributes.txt,81
1,calendar.txt,187
2,feed_info.txt,319
3,farezone_attributes.txt,18
4,areas.txt,12
5,frequencies.txt,53
6,fare_rider_categories.txt,32
7,calendar_dates.txt,544
8,agency.txt,224
9,rider_categories.txt,45


In [60]:
## Compute the trip stats
trip_stats=feed.compute_trip_stats()
trip_stats.head().T

,60,55,23,21,37
trip_id,t_18939_b_642_tn_0,t_18934_b_642_tn_0,t_18902_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0
route_id,3231,3231,3231,3232,3232
route_short_name,The Rev,The Rev,The Rev,The Rev,The Rev
route_type,3,3,3,3,3
direction_id,NaN,NaN,NaN,NaN,NaN
shape_id,p_6746,p_6760,p_6759,p_6758,p_6745
num_stops,13,14,13,12,14
start_time,06:30:00,07:15:00,08:30:00,16:10:00,16:57:00
end_time,07:15:00,08:30:00,09:40:00,16:57:00,18:25:00
start_stop_id,807645,807693,807693,807691,807645


In [62]:
# Add shape_dist_traveled column to stop times
feed = feed.append_dist_to_stop_times()
feed.stop_times.head().T

,0,1,2,3,4
trip_id,t_18879_b_642_tn_0,t_18879_b_642_tn_0,t_18879_b_642_tn_0,t_18879_b_642_tn_0,t_18879_b_642_tn_0
arrival_time,19:28:00,19:30:00,NaN,NaN,NaN
departure_time,19:28:00,19:30:00,NaN,NaN,NaN
stop_id,835935,835940,835939,835938,835937
stop_sequence,1,2,3,4,5
stop_headsign,NaN,NaN,NaN,NaN,NaN
pickup_type,2,0,2,2,2
drop_off_type,2,0,2,2,2
timepoint,0,0,0,0,0
continuous_stops,0,0,0,0,0


In [64]:
## Choose the study dates 
week=feed.get_first_week()
date=[week[0],week[4]] ##  is counted as sunday and 4-wdnesday
date

['20151102', '20151106']

In [65]:
## COmpute the feed time series
fts=feed.compute_feed_time_series(trip_stats,date,freq='5H')

In [66]:
fts

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2015-11-02 00:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 05:00:00,31.0,37.0,37.0,880298.100479,31.700000,27769.656167
2015-11-02 10:00:00,8.0,2.0,8.0,80035.151265,2.900000,27598.328022
2015-11-02 15:00:00,34.0,35.0,35.0,850565.510858,31.716667,26817.619890
2015-11-02 20:00:00,1.0,0.0,1.0,0.000000,0.000000,0.000000
2015-11-03 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-03 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-03 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-03 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
gk.downsample(fts,freq='12H')

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2015-11-02 00:00:00,39.0,39.0,39.0,960333.251744,34.600000,27755.296293
2015-11-02 12:00:00,35.0,35.0,35.0,850565.510858,31.716667,26817.619890
2015-11-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-03 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-04 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-05 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Compute feed stats for first week

feed_stats=feed.compute_feed_stats(trip_stats,dates=week)
feed_stats

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151102
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151103
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151104
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151105
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151106


In [54]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, date, freq='12H')
rts

indicator           num_trip_ends       ... service_speed              
route_id                     3231 3232  ...          4766          4767
datetime                                ...                            
2015-11-02 00:00:00           3.0  0.0  ...  10983.177458  21725.128761
2015-11-02 12:00:00           0.0  3.0  ...   9683.802145  12506.370458
2015-11-03 00:00:00           NaN  NaN  ...           NaN           NaN
2015-11-03 12:00:00           NaN  NaN  ...           NaN           NaN
2015-11-04 00:00:00           NaN  NaN  ...           NaN           NaN
2015-11-04 12:00:00           NaN  NaN  ...           NaN           NaN
2015-11-05 00:00:00           NaN  NaN  ...           NaN           NaN
2015-11-05 12:00:00           NaN  NaN  ...           NaN           NaN
2015-11-06 00:00:00           3.0  0.0  ...  10983.177458  21725.128761
2015-11-06 12:00:00           0.0  3.0  ...   9683.802145  12506.370458

[10 rows x 54 columns]

In [55]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['1','2','3']

rts.loc[:, (inds, rids)]

datetime
2015-11-02 00:00:00
2015-11-02 12:00:00
2015-11-03 00:00:00
2015-11-03 12:00:00
2015-11-04 00:00:00
2015-11-04 12:00:00
2015-11-05 00:00:00
2015-11-05 12:00:00
2015-11-06 00:00:00
2015-11-06 12:00:00


In [40]:
feed2=feed.append_dist_to_stop_times()

ValueError: ignored

In [56]:
# Compute trip locations for every hour

rng = pd.date_range('14/12/2018', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(date[0], times)
loc.head()

,trip_id,time,rel_dist,route_id,direction_id,shape_id,lon,lat
0,t_18880_b_642_tn_0,09:00:00,0.612662,4765,0.0,p_8036,-71.210199,42.296657
1,t_18881_b_642_tn_0,20:00:00,NaN,4732,1.0,p_8022,-71.142620,42.395420
2,t_18882_b_642_tn_0,10:00:00,0.919261,4733,0.0,p_6752,-71.260250,42.390311
3,t_18883_b_642_tn_0,09:00:00,0.000000,4734,0.0,p_6754,-71.142620,42.395420
4,t_18884_b_642_tn_0,07:00:00,0.458241,4732,0.0,p_6748,-71.252566,42.416720


In [57]:
# Build a route timetable

route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, date).T

,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,172,173,174,175,176,177,178,179,180,181,182,183,184,185,...,275,276,277,278,279,280,281,267,268,269,270,271,272,273,257,258,259,260,261,262,263,264,265,266,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239
route_id,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,...,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732
service_id,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,...,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31
trip_id,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18884_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18890_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,t_18923_b_642_tn_0,...,t_18892_b_642_tn_0,t_18892_b_642_tn_0,t_18892_b_642_tn_0,t_18892_b_642_tn_0,t_18892_b_642_tn_0,t_18892_b_642_tn_0,t_18892_b_642_tn_0,t_18908_b_642_tn_0,t_18908_b_642_tn_0,t_18908_b_642_tn_0,t_18908_b_642_tn_0,t_18908_b_642_tn_0,t_18908_b_642_tn_0,t_18908_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18889_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0,t_18881_b_642_tn_0
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [58]:
rids = feed.routes.route_id.loc[2:4]
feed.map_routes(rids)

/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


In [67]:
feed.trips.filter(['route_id','trip_id','shape_id']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
route_id,3231,3231,3231,3232,3232,3232,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4733,4733,4733,4733,4733,4733,4733,4733,4733,4733,4734,4734,4734,4734,4734,4734,4734,4734,4734,4736,4736,4736,4736,4736,4736,4736,4736,4736,4765,4765,4765,4765,4765,4765,4765,4765,4765,4766,4766,4766,4766,4766,4766,4767,4767,4767,4767,4767
trip_id,t_18902_b_642_tn_0,t_18939_b_642_tn_0,t_18934_b_642_tn_0,t_18888_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0,t_18952_b_642_tn_0,t_18884_b_642_tn_0,t_18890_b_642_tn_0,t_18907_b_642_tn_0,t_18909_b_642_tn_0,t_18918_b_642_tn_0,t_18919_b_642_tn_0,t_18923_b_642_tn_0,t_18928_b_642_tn_0,t_18950_b_642_tn_0,t_18903_b_642_tn_0,t_18881_b_642_tn_0,t_18949_b_642_tn_0,t_18885_b_642_tn_0,t_18889_b_642_tn_0,t_18908_b_642_tn_0,t_18892_b_642_tn_0,t_18894_b_642_tn_0,t_18896_b_642_tn_0,t_18899_b_642_tn_0,t_18932_b_642_tn_0,t_18882_b_642_tn_0,t_18929_b_642_tn_0,t_18951_b_642_tn_0,t_18941_b_642_tn_0,t_18945_b_642_tn_0,t_18879_b_642_tn_0,t_18931_b_642_tn_0,t_18901_b_642_tn_0,t_18914_b_642_tn_0,t_18883_b_642_tn_0,t_18930_b_642_tn_0,t_18905_b_642_tn_0,t_18895_b_642_tn_0,t_18891_b_642_tn_0,t_18938_b_642_tn_0,t_18911_b_642_tn_0,t_18904_b_642_tn_0,t_18936_b_642_tn_0,t_18948_b_642_tn_0,t_18886_b_642_tn_0,t_18927_b_642_tn_0,t_18935_b_642_tn_0,t_18947_b_642_tn_0,t_18946_b_642_tn_0,t_18887_b_642_tn_0,t_18915_b_642_tn_0,t_18917_b_642_tn_0,t_18913_b_642_tn_0,t_18926_b_642_tn_0,t_18880_b_642_tn_0,t_18897_b_642_tn_0,t_18920_b_642_tn_0,t_18933_b_642_tn_0,t_18893_b_642_tn_0,t_18921_b_642_tn_0,t_18937_b_642_tn_0,t_18925_b_642_tn_0,t_18924_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18906_b_642_tn_0,t_18910_b_642_tn_0,t_18943_b_642_tn_0,t_18898_b_642_tn_0,t_18944_b_642_tn_0,t_18940_b_642_tn_0,t_18912_b_642_tn_0
shape_id,p_6759,p_6746,p_6760,p_6745,p_6758,p_6745,p_6750,p_6748,p_6751,p_6751,p_6748,p_6747,p_6747,p_6748,p_6749,p_6748,p_8019,p_8022,p_8021,p_8017,p_8018,p_8020,p_8016,p_8021,p_8019,p_8017,p_6752,p_6752,p_6753,p_6752,p_6753,p_8025,p_8027,p_8026,p_8028,p_8029,p_6754,p_6755,p_6755,p_6755,p_6754,p_8032,p_8031,p_8032,p_8031,p_6756,p_6757,p_6756,p_6757,p_6757,p_8034,p_8033,p_8034,p_8033,p_8036,p_8036,p_8036,p_8036,p_8035,p_8038,p_8038,p_8038,p_8037,p_8040,p_8040,p_8039,p_8024,p_8023,p_8024,p_8042,p_8041,p_8042,p_8044,p_8043


In [69]:
## FInd all the trips of specific route : slice rows
feed.trips.loc[lambda x : x.route_id == '3231']

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,bikes_allowed,wheelchair_accessible,trip_type
0,3231,c_1725_b_642_d_31,t_18902_b_642_tn_0,NaN,NaN,NaN,3270,p_6759,NaN,NaN,NaN
1,3231,c_1725_b_642_d_31,t_18939_b_642_tn_0,NaN,NaN,NaN,3270,p_6746,NaN,NaN,NaN
2,3231,c_1725_b_642_d_31,t_18934_b_642_tn_0,NaN,NaN,NaN,3270,p_6760,NaN,NaN,NaN


In [70]:
## group all the trips by route 
feed.trips.groupby('route_id').agg(['mean']).T


,route_id,3231,3232,4732,4733,4734,4736,4765,4766,4767
direction_id,mean,NaN,NaN,0.5,0.5,0.444444,0.444444,0.444444,0.5,0.4
bikes_allowed,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wheelchair_accessible,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trip_type,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
feed.trips

In [71]:
## Comute the number of trips per route : split ,apply,combine
def my_agg(group):
  d={}
  d['num_trips']=group.shape[0]
  return pd.Series(d)


## Use method chaning 
(
    feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg)
 .reset_index()
 #Join in route short name from feeed routes
 .merge(feed.routes.filter(['route_id','route_short_name','route_long_name','route_type']))
 #sort values 
 .sort_values('num_trips',ascending=False).T
)

,2,3,4,5,6,7,8,0,1
route_id,4732,4733,4734,4736,4765,4766,4767,3231,3232
num_trips,20,10,9,9,9,6,5,3,3
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev
route_long_name,Alewife Shuttle Route A,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route D,Needham Shuttle,Waltham Shuttle Route A,Waltham Shuttle Route B,Hartwell Area Shuttle (AM),Hartwell Area Shuttle (PM)
route_type,3,3,3,3,3,3,3,3,3


In [72]:
(feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg))

,num_trips
route_id,
3231,3
3232,3
4732,20
4733,10
4734,9
4736,9
4765,9
4766,6
4767,5


In [73]:
feed.trips.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
route_id,3231,3231,3231,3232,3232,3232,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4733,4733,4733,4733,4733,4733,4733,4733,4733,4733,4734,4734,4734,4734,4734,4734,4734,4734,4734,4736,4736,4736,4736,4736,4736,4736,4736,4736,4765,4765,4765,4765,4765,4765,4765,4765,4765,4766,4766,4766,4766,4766,4766,4767,4767,4767,4767,4767
service_id,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31
trip_id,t_18902_b_642_tn_0,t_18939_b_642_tn_0,t_18934_b_642_tn_0,t_18888_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0,t_18952_b_642_tn_0,t_18884_b_642_tn_0,t_18890_b_642_tn_0,t_18907_b_642_tn_0,t_18909_b_642_tn_0,t_18918_b_642_tn_0,t_18919_b_642_tn_0,t_18923_b_642_tn_0,t_18928_b_642_tn_0,t_18950_b_642_tn_0,t_18903_b_642_tn_0,t_18881_b_642_tn_0,t_18949_b_642_tn_0,t_18885_b_642_tn_0,t_18889_b_642_tn_0,t_18908_b_642_tn_0,t_18892_b_642_tn_0,t_18894_b_642_tn_0,t_18896_b_642_tn_0,t_18899_b_642_tn_0,t_18932_b_642_tn_0,t_18882_b_642_tn_0,t_18929_b_642_tn_0,t_18951_b_642_tn_0,t_18941_b_642_tn_0,t_18945_b_642_tn_0,t_18879_b_642_tn_0,t_18931_b_642_tn_0,t_18901_b_642_tn_0,t_18914_b_642_tn_0,t_18883_b_642_tn_0,t_18930_b_642_tn_0,t_18905_b_642_tn_0,t_18895_b_642_tn_0,t_18891_b_642_tn_0,t_18938_b_642_tn_0,t_18911_b_642_tn_0,t_18904_b_642_tn_0,t_18936_b_642_tn_0,t_18948_b_642_tn_0,t_18886_b_642_tn_0,t_18927_b_642_tn_0,t_18935_b_642_tn_0,t_18947_b_642_tn_0,t_18946_b_642_tn_0,t_18887_b_642_tn_0,t_18915_b_642_tn_0,t_18917_b_642_tn_0,t_18913_b_642_tn_0,t_18926_b_642_tn_0,t_18880_b_642_tn_0,t_18897_b_642_tn_0,t_18920_b_642_tn_0,t_18933_b_642_tn_0,t_18893_b_642_tn_0,t_18921_b_642_tn_0,t_18937_b_642_tn_0,t_18925_b_642_tn_0,t_18924_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18906_b_642_tn_0,t_18910_b_642_tn_0,t_18943_b_642_tn_0,t_18898_b_642_tn_0,t_18944_b_642_tn_0,t_18940_b_642_tn_0,t_18912_b_642_tn_0
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trip_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,NaN,NaN,NaN,NaN,

In [ ]:
## Fill in shape dist travlled columns of stop times for later distance calucltaions 
trip_stats=feed.compute_trip_stats()

In [ ]:
trip_stats.T

In [ ]:
feed.trips

In [75]:
## Route stats
route_stats=feed.compute_route_stats(trip_stats,dates=date)

In [ ]:
feed=feed.append_dist_to_stop_times()
feed.stop_times.T

In [ ]:
 # Append shape_dist_traveled column to stop times

display(feed.stop_times.head().T)

In [ ]:
feed = feed.append_dist_to_stop_times()
feed.stop_times.head().T

In [ ]:
# Choose study dates

week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday
dates

In [ ]:
# Compute feed time series

trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
fts

In [ ]:
gk.downsample(fts, freq='12H')


In [ ]:
# Compute feed stats for first week

feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

In [ ]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, dates, freq='12H')
rts

In [ ]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['110-423', '110N-423']

rts.loc[:, (inds, rids)]

In [ ]:
# Slice again by cross-section

rts.xs(rids[0], axis="columns", level=1)

In [74]:
# Compute trip locations for every hour

rng = pd.date_range('1/1/2000', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(date[0], times)
loc.head()

,trip_id,time,rel_dist,route_id,direction_id,shape_id,lon,lat
0,t_18880_b_642_tn_0,09:00:00,0.612663,4765,0.0,p_8036,-71.210199,42.296657
1,t_18881_b_642_tn_0,20:00:00,NaN,4732,1.0,p_8022,-71.142620,42.395420
2,t_18882_b_642_tn_0,10:00:00,0.919251,4733,0.0,p_6752,-71.260250,42.390313
3,t_18883_b_642_tn_0,09:00:00,0.000000,4734,0.0,p_6754,-71.142620,42.395420
4,t_18884_b_642_tn_0,07:00:00,0.458245,4732,0.0,p_6748,-71.252567,42.416719


In [ ]:
# Build a route timetable

route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

In [ ]:
# Compute screen line counts
DATE='20201206'

# Route stats
route_stats = feed.compute_route_stats(trip_stats, dates=[DATE])
route_stats.T

In [76]:
# Join in route long name
route_stats = route_stats.merge(feed.routes.filter(["route_id", "route_long_name"]))

cols = [
    "route_id",
    "route_short_name", 
    "route_long_name", 
    "route_type", 
    "start_time",
    "end_time",
    "num_trips",
    "max_headway",
    "mean_trip_distance",
    "mean_trip_duration",
    "service_speed",
]

In [77]:
# The most/least frequent routes are
(
    route_stats
    .filter(cols)
    .sort_values("max_headway")
    .T
)

,4,5,12,13,14,15,6,7,8,9,10,11,16,17,0,1,2,3
route_id,4732,4732,4765,4765,4766,4766,4733,4733,4734,4734,4736,4736,4767,4767,3231,3231,3232,3232
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,The Rev,The Rev
route_long_name,Alewife Shuttle Route A,Alewife Shuttle Route A,Needham Shuttle,Needham Shuttle,Waltham Shuttle Route A,Waltham Shuttle Route A,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route D,Alewife Shuttle Route D,Waltham Shuttle Route B,Waltham Shuttle Route B,Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM)
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
start_time,06:30:00,06:30:00,06:30:00,06:30:00,08:30:00,08:30:00,06:45:00,06:45:00,06:45:00,06:45:00,06:10:00,06:10:00,07:15:00,07:15:00,06:30:00,06:30:00,16:10:00,16:10:00
end_time,20:00:00,20:00:00,18:40:00,18:40:00,18:40:00,18:40:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:15:00,19:15:00,09:40:00,09:40:00,19:45:00,19:45:00
num_trips,20,20,9,9,6,6,10,10,9,9,9,9,5,5,3,3,3,3
max_headway,50,50,50,50,70,70,80,80,80,80,80,80,110,110,NaN,NaN,NaN,NaN
mean_trip_distance,24662.2,24662.2,11142.3,11142.3,8288.11,8288.11,27317.3,27317.3,31069.7,31069.7,33684.7,33684.7,17315.4,17315.4,36130,36130,38905.7,38905.7
mean_trip_duration,0.864167,0.864167,0.696296,0.696296,0.805556,0.805556,0.863333,0.863333,1.05185,1.05185,0.888889,0.888889,1.02333,1.02333,1.05556,1.05556,1.18333,1.18333


In [78]:
# The shortest/longest routes are
(
    route_stats
    .filter(cols)
    .sort_values("mean_trip_distance")
    .T
)

,15,14,13,12,17,16,4,5,6,7,8,9,11,10,1,0,3,2
route_id,4766,4766,4765,4765,4767,4767,4732,4732,4733,4733,4734,4734,4736,4736,3231,3231,3232,3232
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,The Rev,The Rev
route_long_name,Waltham Shuttle Route A,Waltham Shuttle Route A,Needham Shuttle,Needham Shuttle,Waltham Shuttle Route B,Waltham Shuttle Route B,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route D,Alewife Shuttle Route D,Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM)
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
start_time,08:30:00,08:30:00,06:30:00,06:30:00,07:15:00,07:15:00,06:30:00,06:30:00,06:45:00,06:45:00,06:45:00,06:45:00,06:10:00,06:10:00,06:30:00,06:30:00,16:10:00,16:10:00
end_time,18:40:00,18:40:00,18:40:00,18:40:00,19:15:00,19:15:00,20:00:00,20:00:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,09:40:00,09:40:00,19:45:00,19:45:00
num_trips,6,6,9,9,5,5,20,20,10,10,9,9,9,9,3,3,3,3
max_headway,70,70,50,50,110,110,50,50,80,80,80,80,80,80,NaN,NaN,NaN,NaN
mean_trip_distance,8288.11,8288.11,11142.3,11142.3,17315.4,17315.4,24662.2,24662.2,27317.3,27317.3,31069.7,31069.7,33684.7,33684.7,36130,36130,38905.7,38905.7
mean_trip_duration,0.805556,0.805556,0.696296,0.696296,1.02333,1.02333,0.864167,0.864167,0.863333,0.863333,1.05185,1.05185,0.888889,0.888889,1.05556,1.05556,1.18333,1.18333


In [79]:
# The slowest/fastest routes are
(
    route_stats
    .filter(cols)
    .sort_values("service_speed")
    .T
)

,15,14,13,12,17,16,4,5,9,8,6,7,3,2,1,0,10,11
route_id,4766,4766,4765,4765,4767,4767,4732,4732,4734,4734,4733,4733,3232,3232,3231,3231,4736,4736
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,The Rev,The Rev,NaN,NaN
route_long_name,Waltham Shuttle Route A,Waltham Shuttle Route A,Needham Shuttle,Needham Shuttle,Waltham Shuttle Route B,Waltham Shuttle Route B,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route B,Alewife Shuttle Route B,Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Alewife Shuttle Route D,Alewife Shuttle Route D
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
start_time,08:30:00,08:30:00,06:30:00,06:30:00,07:15:00,07:15:00,06:30:00,06:30:00,06:45:00,06:45:00,06:45:00,06:45:00,16:10:00,16:10:00,06:30:00,06:30:00,06:10:00,06:10:00
end_time,18:40:00,18:40:00,18:40:00,18:40:00,19:15:00,19:15:00,20:00:00,20:00:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,09:40:00,09:40:00,19:45:00,19:45:00
num_trips,6,6,9,9,5,5,20,20,9,9,10,10,3,3,3,3,9,9
max_headway,70,70,50,50,110,110,50,50,80,80,80,80,NaN,NaN,NaN,NaN,80,80
mean_trip_distance,8288.11,8288.11,11142.3,11142.3,17315.4,17315.4,24662.2,24662.2,31069.7,31069.7,27317.3,27317.3,38905.7,38905.7,36130,36130,33684.7,33684.7
mean_trip_duration,0.805556,0.805556,0.696296,0.696296,1.02333,1.02333,0.864167,0.864167,1.05185,1.05185,0.863333,0.863333,1.18333,1.18333,1.05556,1.05556,0.888889,0.888889


In [80]:
# Map routes

# Map our extreme routes
feed.map_routes(route_ids=['4765','4766'])


/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)
